In [1]:
import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import Sequence

from tensorflow.keras.layers import Dense, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Conv2DTranspose
import matplotlib.pyplot as plt
from keras import backend as K
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.utils import shuffle
import os
import cv2
from tensorflow.keras.callbacks import Callback

from tensorflow.keras.models import Model

Using TensorFlow backend.


In [2]:
img_w,img_h = 224,224

inputs1 = Input(shape=(img_h, img_w, 3,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(inputs1)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(3, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=inputs1, outputs=decoder_output)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
# summarize model
print(model.summary())
# plot_model(model, to_file='autoencoder_colorization_baseline.png', show_shapes=True)
    
model.compile(optimizer='adamax',loss='mse')

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 128)       147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 256)       590080    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 512)      

In [3]:
before = 'before_filtering/'
after = 'after_filtering/'
befores = os.listdir(before)
afters = os.listdir(after)
print(len(befores),len(afters))

X = [];Y = []
for i in range(len(befores)):
    x = cv2.imread(before+befores[i])
    x = cv2.resize(x,(224,224))
    x=x/255
    X.append(x)

    y = cv2.imread(after+befores[i])
    y = cv2.resize(y,(224,224))
    y=y/255
    Y.append(y)


204 204


In [4]:
X = np.array(X)
Y = np.array(Y)
X.shape, Y.shape

((204, 224, 224, 3), (204, 224, 224, 3))

In [5]:
h = model.fit(x=X, y=Y,
              batch_size = 8,
              epochs = 200)

Epoch 1/200
26/26 [==============================] - 29s 1s/step - loss: 0.0956
Epoch 2/200
26/26 [==============================] - 29s 1s/step - loss: 0.0439
Epoch 3/200
26/26 [==============================] - 29s 1s/step - loss: 0.0371
Epoch 4/200
26/26 [==============================] - 30s 1s/step - loss: 0.0304
Epoch 5/200
26/26 [==============================] - 30s 1s/step - loss: 0.0271
Epoch 6/200
26/26 [==============================] - 30s 1s/step - loss: 0.0237
Epoch 7/200
26/26 [==============================] - 30s 1s/step - loss: 0.0210
Epoch 8/200
26/26 [==============================] - 30s 1s/step - loss: 0.0194
Epoch 9/200
26/26 [==============================] - 30s 1s/step - loss: 0.0178
Epoch 10/200
26/26 [==============================] - 30s 1s/step - loss: 0.0164
Epoch 11/200
26/26 [==============================] - 30s 1s/step - loss: 0.0159
Epoch 12/200
26/26 [==============================] - 30s 1s/step - loss: 0.0151
Epoch 13/200
26/26 [=================

26/26 [==============================] - 31s 1s/step - loss: 0.0062
Epoch 103/200
26/26 [==============================] - 31s 1s/step - loss: 0.0061
Epoch 104/200
26/26 [==============================] - 31s 1s/step - loss: 0.0060
Epoch 105/200
26/26 [==============================] - 31s 1s/step - loss: 0.0061
Epoch 106/200
26/26 [==============================] - 31s 1s/step - loss: 0.0063
Epoch 107/200
26/26 [==============================] - 31s 1s/step - loss: 0.0062
Epoch 108/200
26/26 [==============================] - 31s 1s/step - loss: 0.0059
Epoch 109/200
26/26 [==============================] - 31s 1s/step - loss: 0.0058
Epoch 110/200
26/26 [==============================] - 30s 1s/step - loss: 0.0058
Epoch 111/200
26/26 [==============================] - 29s 1s/step - loss: 0.0062
Epoch 112/200
26/26 [==============================] - 29s 1s/step - loss: 0.0064
Epoch 113/200
26/26 [==============================] - 29s 1s/step - loss: 0.0064
Epoch 114/200
26/26 [=========

KeyboardInterrupt: 

In [6]:
model.save("model_175_epochs.h5")

In [7]:
plt.plot(h.history['loss'])
plt.xlabel('Epoch')
plt.ylabel('MSE loss')
plt.show()

NameError: name 'h' is not defined